CTDC014 ctDNA Analysis 

How this patient was selected?

Using the command - 

(echo -e "PatientID\tSample Name\tIsolate\tTreatment Stage\tExperiment"; csvcut -c "Sample Name","isolate","Experiment" ~/Downloads/SraRunTable.csv | sed '1d' | awk -F',' '{
  if ($1 ~ /^PBMC_/) stage="Pre-treatment";
  else if ($1 ~ /^FFPE_/) stage="Post-treatment";
  else if ($1 ~ /-[0-9]+$/) stage="During treatment";
  else stage="Baseline";
  split($1, a, /[^0-9]/);
  for (i in a) if (a[i] ~ /^[0-9]+$/) {id=a[i]; break}
  print id "\t" $1 "\t" $2 "\t" stage "\t" $3
}' | sort -k1,1n | grep '^014')

Output \n

PatientID       Sample Name     Isolate Treatment Stage Experiment \n

014     C_MB014-2       blood   During treatment        SRX10351577 \n

014     C_MB014-3       blood   During treatment        SRX10351578 \n

014     FFPE_C_MB014    tissue biopsy   Post-treatment  SRX10351726 \n

014     PBMC_C_MB014    blood   Pre-treatment   SRX10351683 \n

014     PBMC_C_MB014-2  blood   Pre-treatment   SRX10351672 \n

Metadata file: SraRunTable.csv was downloaded from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA714799&o=acc_s%3Aa by clicking 'Metadata' under 'Download'

CTDC014 in Table 4 in the paper - https://www.nature.com/articles/s41598-021-95345-4/tables/4
has Gene KRAS missense variant with Pathogenic/Likely_pathogenic 



Download raw sequencing files 

for srr in SRR21014913 SRR21014914 SRR21015062 SRR21015019 SRR21015008; do prefetch "$srr" && fasterq-dump "$srr" --split-files --outdir /Volumes/T7/GTMBioinfo/raw; done

QC of sequencing files

commands of fastqc and multiqc used

    fastqc /Volumes/T7/GTMBioinfo/raw/*.fastq -o docs/

    multiqc docs/ -o docs/

    Save the multiqc report as PDF in docs/ folder from html report

multi qc report - https://github.com/justin-mbca/GTMBioinfo/blob/main/docs/MultiQCReport.pdf
From the report, the quality scores are good in overall and there is high adapter content for a few samples.
    Therefore, trimming is performed before alignment.

    Command - 
 
    mkdir -p /Volumes/T7/GTMBioinfo/trimmed

    for id in SRR21014913 SRR21014914 SRR21015062 SRR21015019 SRR21015008; do
        fastp \
            -i /Volumes/T7/GTMBioinfo/raw/${id}_1.fastq \
            -I /Volumes/T7/GTMBioinfo/raw/${id}_2.fastq \
            -o /Volumes/T7/GTMBioinfo/trimmed/${id}_1.trimmed.fastq \
            -O /Volumes/T7/GTMBioinfo/trimmed/${id}_2.trimmed.fastq \
            --detect_adapter_for_pe \
            --html /Volumes/T7/GTMBioinfo/qc/${id}_fastp.html \
            --json /Volumes/T7/GTMBioinfo/qc/${id}_fastp.json
        done

The duplicate rates are medium high across samples.
    Therefore, the duplicates will be removed during alignment and variant calling steps.

Alignment 
# Create GRCh38 reference genome
   1. Gene KRAS is in Chromosome 12 - https://www.ncbi.nlm.nih.gov/gene?term=(KRAS%5BGene%20Name%5D)%20AND%20Homo%20sapiens%5BOrganism%5D
      1.There for use chromosome 12 as reference of mapping and variant calling to analyze gene KRAS 
   2. Download chromosome 12 of reference fasta file - wget -P /Volumes/T7/GTMBioinfo/reference/ ftp://ftp.ensembl.org/pub/release-115/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
   3. Extract zip file -  gunzip /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
   4. Create BWA index file - bwa index /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa
   5. Create FASTA index - samtools faidx /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa
# Alignment
   1. create bash script in alignment_14.sh file 
   2. Loop through each sample's fastq files to perform alignment and bam conversion/indexing
      1. bwa mem is used for mapping medium length PE reads. 
      2. bam binamry file, sorting and indexing for high performance required by downstream steps. 
   3. Checking mapped reads 
      1. samples=("C_MB014-2" "C_MB014-3" "FFPE_C_MB014" "PBMC_C_MB014" "PBMC_C_MB014-2")
      2. for s in "${samples[@]}"; do   bam="/Volumes/T7/GTMBioinfo/aligned/${s}.chr12.sorted.bam";   echo " Checking $s";   samtools idxstats "$bam";   echo ""; done
   
            Checking C_MB014-2
            12      133275309       215463  25540 (chromosome 12, chromosome length, mapped reads, unmapped reads)
            *       0       0       513128

            Checking C_MB014-3
            12      133275309       316406  34464
            *       0       0       745550

            Checking FFPE_C_MB014
            12      133275309       308192  30161
            *       0       0       659328

            Checking PBMC_C_MB014
            12      133275309       331136  39013
            *       0       0       811528

            Checking PBMC_C_MB014-2
            12      133275309       401130  46824
            *       0       0       971682


In [ ]:
Variant calling

script variant_calling.py
script variant_calling_lowerthreshold.py  -- with relaxing thresholds to check if variants can be found. 

---check variant results

cd /Volumes/T7/GTMBioinfo/variants

KRAS_REGION="12:25245350-25250930"

for s in C_MB014-2 C_MB014-3 FFPE_C_MB014 PBMC_C_MB014 PBMC_C_MB014-2; do
  echo "$s — Filtered VCF:"
  bcftools view -r $KRAS_REGION /Volumes/T7/GTMBioinfo/variants/${s}.filtered.vcf.gz | grep -v "^#" || echo "No variants found in filtered VCF"
  echo ""

  echo "$s — Unfiltered VCF:"
  bcftools view -r $KRAS_REGION /Volumes/T7/GTMBioinfo/variants/${s}.unfiltered.vcf.gz | grep -v "^#" || echo "No variants found in unfiltered VCF"
  echo "------------------------------------------------------------"
done

for s in C_MB014-2 C_MB014-3 FFPE_C_MB014 PBMC_C_MB014 PBMC_C_MB014-2; do
>   echo "$s — Filtered VCF:"
>   bcftools view -r $KRAS_REGION /Volumes/T7/GTMBioinfo/variants/${s}.filtered.vcf.gz | grep -v "^#" || echo "No variants found in filtered VCF"
>   echo ""
> 
>   echo "$s — Unfiltered VCF:"
>   bcftools view -r $KRAS_REGION /Volumes/T7/GTMBioinfo/variants/${s}.unfiltered.vcf.gz | grep -v "^#" || echo "No variants found in unfiltered VCF"
>   echo "------------------------------------------------------------"
> done
C_MB014-2 — Filtered VCF:
No variants found in filtered VCF

C_MB014-2 — Unfiltered VCF:
No variants found in unfiltered VCF
------------------------------------------------------------
C_MB014-3 — Filtered VCF:
No variants found in filtered VCF

C_MB014-3 — Unfiltered VCF:
No variants found in unfiltered VCF
------------------------------------------------------------
FFPE_C_MB014 — Filtered VCF:
No variants found in filtered VCF

FFPE_C_MB014 — Unfiltered VCF:
No variants found in unfiltered VCF
------------------------------------------------------------
PBMC_C_MB014 — Filtered VCF:
12      25250133        .       G       A       .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=53;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250135        .       GG      AA      .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=51;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250144        .       G       A       .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,25;MFRL=0,0;MMQ=60,48;MPOS=42;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250156        .       G       A       .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=30;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250165        .       A       C       .       base_qual;clustered_events;haplotype    AS_FilterStatus=base_qual;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,11;MFRL=0,0;MMQ=60,48;MPOS=21;POPAF=7.3;TLOD=4.2   GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,0:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250169        .       G       A       .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=17;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250171        .       G       A       .       clustered_events;haplotype      AS_FilterStatus=SITE;AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;GERMQ=40;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=15;POPAF=7.3;TLOD=4.2        GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB     0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0

PBMC_C_MB014 — Unfiltered VCF:
12      25250133        .       G       A       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=53;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250135        .       GG      AA      .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=51;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250144        .       G       A       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,25;MFRL=0,0;MMQ=60,48;MPOS=42;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250133_G_A:25250133:0,0,1,0
12      25250156        .       G       A       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=30;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250165        .       A       C       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,11;MFRL=0,0;MMQ=60,48;MPOS=21;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,0:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250169        .       G       A       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=17;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
12      25250171        .       G       A       .       .       AS_SB_TABLE=0,0|0,0;DP=1;ECNT=7;ECNTH=7;MBQ=0,37;MFRL=0,0;MMQ=60,48;MPOS=15;POPAF=7.3;TLOD=4.2      GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB 0|1:0,1:0.667:1:0,1:0,0:0,1:0|1:25250156_G_A:25250156:0,0,1,0
------------------------------------------------------------
PBMC_C_MB014-2 — Filtered VCF:
No variants found in filtered VCF

PBMC_C_MB014-2 — Unfiltered VCF:
No variants found in unfiltered VCF
------------------------------------------------------------


Interpret mapping in PBMC_C_MB014 - DP=1 : total depth - only 1 read supports this site mutation, which cannot support variant statistically significant. 

IGV screenshot - https://github.com/justin-mbca/GTMBioinfo/blob/main/docs/igv_snapshot.png to inspect sample PBMC_C_MB014 mapping and vcf. 

  - all these sample has not variants found. 


Conclusion and Interpretation: 

  - No true variants were called in filtered or relaxed VCFs

  - Unfiltered artifacts in PBMC_C_MB014 were:

    - Single-read

    - Low base quality

    - Not supported in IGV

    - no evidence of clonal or subclonal KRAS mutation

    - This conclusion is not aligning against the result in the paper 

        CTDC014 in Table 4 in the paper - https://www.nature.com/articles/s41598-021-95345-4/tables/4
        has Gene KRAS missense variant with Pathogenic/Likely_pathogenic 